In [1]:
# %%
from ProtDiffusion.training_utils import VAETrainingConfig, make_clustered_dataloader, set_seed, VAETrainer, count_parameters
from transformers import PreTrainedTokenizerFast

from datasets import load_from_disk

from ProtDiffusion.models.autoencoder_kl_1d import AutoencoderKL1D

import os

seed = 42
set_seed(seed) # Set the random seed for reproducibility

dataset = load_from_disk('/home/kkj/ProtDiffusion/datasets/UniRef50_grouped')
# dataset = load_from_disk('/home/kkj/ProtDiffusion/datasets/ACP-bad?_grouped')
dataset = dataset.shuffle(seed)

# %%
# tokenizer = PreTrainedTokenizerFast.from_pretrained("/zhome/fb/0/155603/ProtDiffusion/ProtDiffusion/tokenizer/tokenizer_v4.1")
# tokenizer = PreTrainedTokenizerFast.from_pretrained("/home/kkj/ProtDiffusion/ProtDiffusion/tokenizer/tokenizer_v4.1")
tokenizer = PreTrainedTokenizerFast.from_pretrained("/home/kkj/ProtDiffusion/ProtDiffusion/tokenizer/tokenizer_v4.2")

# Split the dataset into train and temp sets using the datasets library
train_test_split_ratio = 0.0002
train_val_test_split = dataset.train_test_split(test_size=train_test_split_ratio, seed=seed)
train_dataset = train_val_test_split['train']
temp_dataset = train_val_test_split['test']

# Split the temp set into validation and test sets using the datasets library
val_test_split_ratio = 0.5
val_test_split = temp_dataset.train_test_split(test_size=val_test_split_ratio, seed=seed)
val_dataset = val_test_split['train']
test_dataset = val_test_split['test']

# Check dataset lengths
print(f"Train dataset length: {len(train_dataset)}")
print(f"Validation dataset length: {len(val_dataset)}")
print(f"Test dataset length: {len(test_dataset)}")

Random seed set as 42


Loading dataset from disk:   0%|          | 0/149 [00:00<?, ?it/s]

Train dataset length: 42803651
Validation dataset length: 4281
Test dataset length: 4282


In [3]:
mega_batch = 100
batch_size = 16
max_len_start = 64
max_len = max_len_start
pad_to_multiple_of = 8

print("num cpu cores:", os.cpu_count())
print("setting num_workers to 16")
num_workers = 16

train_dataloader = make_clustered_dataloader(batch_size,
                                             mega_batch,
                                             train_dataset,
                                             tokenizer=tokenizer,
                                             max_len=max_len_start,
                                             num_workers=num_workers,
                                             pad_to_multiple_of=pad_to_multiple_of,
)
val_dataloader = make_clustered_dataloader(batch_size,
                                           mega_batch,
                                           val_dataset, 
                                           tokenizer=tokenizer,
                                           max_len=max_len, 
                                           num_workers=1,
                                           pad_to_multiple_of=pad_to_multiple_of,
)
test_dataloader = make_clustered_dataloader(batch_size,
                                            mega_batch,
                                            test_dataset,
                                            tokenizer=tokenizer,
                                            max_len=max_len, 
                                            num_workers=1,
                                            pad_to_multiple_of=pad_to_multiple_of,
)
print("length of train dataloader: ", len(train_dataloader))
print("length of val dataloader: ", len(val_dataloader))
print("length of test dataloader: ", len(test_dataloader))

num cpu cores: 16
setting num_workers to 16
length of train dataloader:  2675229
length of val dataloader:  268
length of test dataloader:  268


In [4]:
for i,batch in enumerate(train_dataloader):
    print(batch)
    if i == 10:
        break

{'id': array([b'UniRef50_A0A376GFI7', b'UniRef50_Q2U3Y4', b'UniRef50_A0A9K3CZR1',
       b'UniRef50_A0A6J8DZ84', b'UniRef50_A0A1Y1XSN8',
       b'UniRef50_A0A0C2MSM0', b'UniRef50_A0A3Q7ETF5', b'UniRef50_G4UIJ3',
       b'UniRef50_A0AA88J1M0', b'UniRef50_A0A6A5KNV8',
       b'UniRef50_A0A2D9SZV8', b'UniRef50_A0A3Q7NKJ5',
       b'UniRef50_A0A1R1YDB7', b'UniRef50_A0A2M7W0D7',
       b'UniRef50_A0A2H0YLB8', b'UniRef50_A0A940L544'], dtype='|S19'), 'label': tensor([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0]), 'length': tensor([176, 176, 176, 176, 176, 176, 176, 176, 176, 176, 176, 176, 176, 176,
        176, 176]), 'input_ids': tensor([[19, 14,  9,  ...,  2, 13, 10],
        [18, 17,  2,  ..., 16, 17, 18],
        [ 7, 19,  3,  ..., 17,  7,  9],
        ...,
        [22, 12, 17,  ...,  5,  5,  6],
        [12, 16,  9,  ...,  0,  0,  0],
        [ 7, 16, 10,  ..., 16,  5, 16]]), 'attention_mask': tensor([[ True,  True,  True,  ...,  True,  True,  True],
        [ True,  True,  True,  ..

In [ ]:
for epoch in range(1):
    for i,batch in enumerate(train_dataloader):
        print("batch", (epoch*len(train_dataloader)) + i)